# **Shopee Code League - Order Brushing**

This notebook is for Shopee Code League Competition, Order Brushing, to detecting abnormal user behaviour. The competition details can be obtained from [here](https://www.kaggle.com/c/order-brushing-shopee-code-league/overview).

Team name: CY Team (Malaysia)

Team member:
1. Chong Li Chuin
2. Ee Yeo Keat

# **Import libraries and read dataset**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time
from datetime import datetime
from datetime import timedelta
from google.colab import files

In [ ]:
from tensorflow.python.client import device_lib
check_device = device_lib.list_local_devices()
check_device

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 7962389366130422802, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 974146674455461370
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1930007338866718707
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11150726272
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 10260854513767219687
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [ ]:
data = pd.read_csv('order_brush_order.csv', index_col=0).sort_values(by=['event_time','shopid','userid']).reset_index()
data

,orderid,shopid,userid,event_time
0,31075200506751,6042309,97707522,2019-12-27 00:00:00
1,31075200506753,8715449,97707522,2019-12-27 00:00:00
2,31075200506752,104804492,97707522,2019-12-27 00:00:00
3,31075201870570,190969466,170182475,2019-12-27 00:00:02
4,31075205798264,2859407,12532131,2019-12-27 00:00:05
...,...,...,...,...
222745,31507183252446,149254894,193333760,2019-12-31 23:59:43
222746,31507187390691,147941492,40258063,2019-12-31 23:59:47
222747,31507191066627,154074176,2338306,2019-12-31 23:59:51
222748,31507191066628,187123853,2338306,2019-12-31 23:59:51


In [ ]:
date = data['event_time']
date_time = []
for i in date:
  date_time.append(datetime.strptime(i,"%Y-%m-%d %H:%M:%S"))

In [ ]:
data['date_time'] = date_time
data

,orderid,shopid,userid,event_time,date_time
0,31075200506751,6042309,97707522,2019-12-27 00:00:00,2019-12-27 00:00:00
1,31075200506753,8715449,97707522,2019-12-27 00:00:00,2019-12-27 00:00:00
2,31075200506752,104804492,97707522,2019-12-27 00:00:00,2019-12-27 00:00:00
3,31075201870570,190969466,170182475,2019-12-27 00:00:02,2019-12-27 00:00:02
4,31075205798264,2859407,12532131,2019-12-27 00:00:05,2019-12-27 00:00:05
...,...,...,...,...,...
222745,31507183252446,149254894,193333760,2019-12-31 23:59:43,2019-12-31 23:59:43
222746,31507187390691,147941492,40258063,2019-12-31 23:59:47,2019-12-31 23:59:47
222747,31507191066627,154074176,2338306,2019-12-31 23:59:51,2019-12-31 23:59:51
222748,31507191066628,187123853,2338306,2019-12-31 23:59:51,2019-12-31 23:59:51


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222750 entries, 0 to 222749
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   orderid     222750 non-null  int64         
 1   shopid      222750 non-null  int64         
 2   userid      222750 non-null  int64         
 3   event_time  222750 non-null  object        
 4   date_time   222750 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 8.5+ MB


In [ ]:
b = data['date_time'].loc[10]
a = data['date_time'].loc[1]
delt = b-a
delt.total_seconds()

8.0

In [ ]:
test = data[:100]
test

,orderid,shopid,userid,event_time,date_time
0,31075200506751,6042309,97707522,2019-12-27 00:00:00,2019-12-27 00:00:00
1,31075200506753,8715449,97707522,2019-12-27 00:00:00,2019-12-27 00:00:00
2,31075200506752,104804492,97707522,2019-12-27 00:00:00,2019-12-27 00:00:00
3,31075201870570,190969466,170182475,2019-12-27 00:00:02,2019-12-27 00:00:02
4,31075205798264,2859407,12532131,2019-12-27 00:00:05,2019-12-27 00:00:05
...,...,...,...,...,...
95,31075285509505,50968872,162009161,2019-12-27 00:01:25,2019-12-27 00:01:25
96,31075285632909,147627088,180684640,2019-12-27 00:01:25,2019-12-27 00:01:25
97,31075288172810,116673348,80101689,2019-12-27 00:01:28,2019-12-27 00:01:28
98,31075289043119,167354126,55074114,2019-12-27 00:01:29,2019-12-27 00:01:29


In [12]:
initial = data['date_time'].loc[1]
it = initial.time()
it + 1

TypeError: ignored

In [ ]:
def check_concentrate_rate():
  concentrate_rate = #Number of Orders within 1 hour / Number of Unique Buyers within 1 hour
  if concentrate_rate >= 3:
    shop_list.append(shopid)
  
  return shop_list

In [ ]:
df.to_csv('CY_Team_submission4.csv') 
files.download('CY_Team_submission4.csv')